In [121]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

import skimage.io as io
import pickle

import time

from sklearn.utils import shuffle

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

In [5]:
path="../GlyphDataset/Dataset/Manual/Preprocessed/"

def loadData(folderPictures=path):
    
    folders=next(os.walk(folderPictures))[1]
    img_groups = {}
    img_list={}

    for folder in folders:
        for img_file in os.listdir(folderPictures+folder):
            name, label = img_file.strip('.png').split("_")
            
            
            # One image per class

            #if label not in img_groups.keys():
            #    img_groups[label] = [folder + "_" + name]


            # Multiple images per class

            if label in img_groups.keys():
                img_groups[label].append(folder+"_"+name)
            else:
                img_groups[label] = [folder+"_"+name]

            img_list[folder+"_"+name]=[label]


    # Remove class with only one hieroglyph


    for k,v in list(img_groups.items()):
        if len(v)==1: del img_groups[k]

    # Extract only N hieroglyph classes randomly

    nclass = len(img_groups.keys())

    list_of_class = random.sample(list(img_groups.keys()), nclass)
#     print(list_of_class)

    short_dico = {x: img_groups[x] for x in list_of_class if x in img_groups}

    dataHiero=pd.DataFrame.from_dict(img_list,orient='index')
    dataHiero.columns = ["label"]
    dataHiero = dataHiero[dataHiero.label != 'UNKNOWN']

    dataHiero = dataHiero.loc[dataHiero['label'].isin(short_dico)]


    dataHiero.reset_index(level=0, inplace=True)

    return dataHiero,img_groups

In [6]:
def read_images(img_groups,path):
    X=[]
    y=[]
    glyph_sizes={}
    low=0
    for glyph in img_groups:
        category_images=[]
        high=low
        for img_path in img_groups[glyph] :
            folder,name = img_path.split('_')
            image = io.imread(path+folder+'/'+name+'_'+glyph+'.png')
            X.append(image)
            y.append(glyph)
            high+=1
#         X.append(np.array(category_images))
        glyph_sizes[glyph]=(low,high-1)
        low=high
        
    return np.array(X),np.array(y).reshape((-1,1)),glyph_sizes
            
    

In [7]:
dataHiero,img_groups=loadData(folderPictures=path)
dataHiero.head()
# img_groups

,index,label
0,39_390115,D21
1,39_390082,M17
2,39_390292,V31
3,39_390375,U15
4,39_390175,D35


In [8]:
X,y,sizes=read_images(img_groups,path)

In [10]:
type(X)
print(y.shape)
print(X.shape)
sizes['D21'][1]

(3711, 1)
(3711, 75, 50)


162

In [12]:
#saving data as pickle
with open("train.pickle", "wb") as f:
    pickle.dump((X,y,sizes),f)

In [13]:
def initialize_weights(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [14]:
def initialize_bias(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [15]:
def hieroRecoModel_offline(input_shape):
    """
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # First Block
    X = Conv2D(64, (3, 3), strides=(2, 2), name='conv1')(X)
    X = BatchNormalization(axis=1, name='bn1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=2)(X)

    X = Conv2D(64, (3, 3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=2)(X)

    X = Flatten()(X)
    X = Dense(128, name='dense_layer')(X)

    # L2 normalization
    X = Lambda(lambda x: K.l2_normalize(x, axis=1))(X)

    features = Model(X_input, X, name="features")

    # Inputs of the siamese network

#     anchor = Input(shape=input_shape)
#     positive = Input(shape=input_shape)
#     negative = Input(shape=input_shape)

#     # Embedding Features of input

#     anchor_features = features(anchor)
#     pos_features = features(positive)
#     neg_features = features(negative)

#     input_triplet = [anchor, positive, negative]
#     output_features = [anchor_features, pos_features, neg_features]

#     # Define the trainable model
#     loss_model = Model(inputs=input_triplet, outputs=output_features,name='loss')
#     loss_model.add_loss(K.mean(triplet_loss(output_features)))
#     loss_model.compile(loss=None,optimizer='adam')


    # Create model instance
    #model = Model(inputs=X_input, outputs=X, name='HieroRecoModel_off')

    return features

In [19]:
model = hieroRecoModel_offline((50, 75, 1))
model.summary()

Model: "features"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 50, 75, 1)         0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 56, 81, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 27, 40, 64)        640       
_________________________________________________________________
bn1 (BatchNormalization)     (None, 27, 40, 64)        108       
_________________________________________________________________
activation_3 (Activation)    (None, 27, 40, 64)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 19, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 17, 64)        369

In [20]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (3,3), activation='relu', input_shape=input_shape, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu',  kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3)))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [21]:
model = get_siamese_model((75, 50, 1))
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 75, 50, 1)    0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 75, 50, 1)    0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 4096)         4891712     input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 4096)         0           sequential_2[1][0]         

In [22]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [102]:
def createPairs(X,y,sizes,batch_size):
    ##create a batch with half it's size are similar glyphs and the other half are different.
    n=0
    i=0
#     input1=[]
#     input2=[]
    label=[]
    _,w,h=X.shape
    # initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, w, h,1)) for i in range(2)]
    
    while n < batch_size:
        random_key1=random.choice(list(sizes))
        low=sizes[random_key1][0]
        high=sizes[random_key1][1]
        index1, index2, index3=np.random.choice(range(low, high), size=3,replace=True)
        random_key2=random.choice(list(sizes))
        
        while random_key2 == random_key1:
            random_key2=random.choice(list(sizes))
            
        low=sizes[random_key2][0]
        high=sizes[random_key2][1]
        index4=np.random.choice(range(low,high))
        n += 2
        # appending images 1 and 3 into input1 and input2 corresponding to y=1 
        #and images 2 and 4 corresponding to y=0
    
        pairs[0][i,:,:,:] = X[index1].reshape( w , h, 1)
        pairs[0][i+1,:,:,:] = X[index2].reshape(w, h, 1)
        pairs[1][i,:,:,:] = X[index3].reshape(w, h, 1)
        pairs[1][i+1,:,:,:] = X[index4].reshape(w, h, 1)
        i += 2
#         input1+=[X[index1],X[index2]]
#         input2+=[X[index3],X[index4]]
        label+=[1,0]
        
#         print(index1,index2,index3,index4)
#         print(y[index1],y[index2],y[index3],y[index4])
#         print(random_key1,random_key2)
    return pairs,label
pairs,label=createPairs(X,y,sizes,32)   

In [162]:
# mm=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
# # np.delete(mm,range(3,6))

In [149]:
def make_oneshot_task(X,y,sizes,N):
    _,w,h=X.shape
    pairs=[np.zeros((N, w, h,1)) for i in range(2)]
    
    true_key=random.choice(list(sizes))
    low=sizes[true_key][0]
    high=sizes[true_key][1]
    
    index=np.random.choice(range(low, high))
    index2=np.random.choice(range(low, high))
    
    test_image= np.asarray([X[index]]*N).reshape(N, w, h,1)
#     print(X.shape)
    X_diff= np.delete(X,range(low,high),axis=0)
#     print(X_diff.shape)
    indices= np.random.choice(range(0, len(X_diff)), size = N-1)
    
    support_set=X_diff[indices,:,:]
    ##!!! adding the similar image to the start of the array is not working!!!!!!!!!!!!!!!!!!!!!
    
#     print(support_set.shape)
#     support_set = [ X[index2] ] + support_set
    support_set=np.insert(support_set,0,X[index2],axis=0)
#     print(support_set.shape)
    targets = np.zeros((N,))
    targets[0] = 1
    support_set=support_set.reshape(N,w,h,1)
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]

    return pairs, targets
    


In [161]:
pp,tt=make_oneshot_task(Xval,yval,sizes_val,20) 
pp[0].shape

(20, 75, 50, 1)

In [156]:
# plt.imshow(pairs[0][0])
pairs[1].shape
# len(label)

(32, 75, 50, 1)

In [99]:
##validation data
val_path="../GlyphDataset/Dataset/Manual/val/"
dataHiero_val,img_groups_val=loadData(folderPictures=val_path)
Xval,yval,sizes_val=read_images(img_groups_val,val_path)
pairsval,labelval=createPairs(Xval,yval,sizes_val,32) 

In [97]:
# def make_oneshot_task(N,X,y):
#     """Create pairs of test image, support set for testing N way one-shot learning. """
#     if s == 'train':
#         X = Xtrain
#         categories = train_classes
#     else:
#         X = Xval
#         categories = val_classes
#     n_classes, n_examples, w, h = X.shape
    
#     indices = rng.randint(0, n_examples,size=(N,))
#  # if no language specified just pick a bunch of random letters
#     categories = rng.choice(range(n_classes),size=(N,),replace=False)            
#     true_category = categories[0]
#     ex1, ex2 = rng.choice(n_examples,replace=False,size=(2,))
#     test_image = np.asarray([X[true_category,ex1,:,:]]*N).reshape(N, w, h,1)
#     support_set = X[categories,indices,:,:]
#     support_set[0,:,:] = X[true_category,ex2]
#     support_set = support_set.reshape(N, w, h,1)
#     targets = np.zeros((N,))
#     targets[0] = 1
#     targets, test_image, support_set = shuffle(targets, test_image, support_set)
#     pairs = [test_image,support_set]

#     return pairs, targets

In [104]:
def test_oneshot(model,X,y,sizes, N, k, s = "val", verbose = 0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(X,y,sizes,N)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct

In [152]:
# Hyper parameters
evaluate_every = 200 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 20000 # No. of training iterations 
N_way = 20 # how many classes for testing one-shot tasks
n_val = 250 # how many one-shot tasks to validate on
best = -1

In [106]:
model_path = './weights/'

In [153]:
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
    (inputs,targets) = createPairs(X,y,sizes,batch_size)
    loss = model.train_on_batch(inputs, targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss)) 
        val_acc = test_oneshot(model,Xval,yval,sizes_val, N_way, n_val, verbose=True)
        model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc

Starting training process!
-------------------------------------

 ------------- 

Time for 200 iterations: 1.6976775844891867 mins
Train Loss: 0.6553137898445129
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 50.4% 20 way one-shot learning accuracy 

Current best: 50.4, previous best: -1

 ------------- 

Time for 400 iterations: 3.447393794854482 mins
Train Loss: 0.4699699282646179
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 62.4% 20 way one-shot learning accuracy 

Current best: 62.4, previous best: 50.4

 ------------- 

Time for 600 iterations: 5.224531070391337 mins
Train Loss: 0.4957376718521118
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 53.2% 20 way one-shot learning accuracy 


 ------------- 

Time for 800 iterations: 7.018525993824005 mins
Train Loss: 0.5388809442520142
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average o

Got an average of 61.2% 20 way one-shot learning accuracy 


 ------------- 

Time for 7200 iterations: 68.8921886563301 mins
Train Loss: 0.38855504989624023
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 67.2% 20 way one-shot learning accuracy 


 ------------- 

Time for 7400 iterations: 70.6104100783666 mins
Train Loss: 0.20513571798801422
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 61.6% 20 way one-shot learning accuracy 


 ------------- 

Time for 7600 iterations: 72.33518684705099 mins
Train Loss: 0.24481472373008728
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 66.4% 20 way one-shot learning accuracy 


 ------------- 

Time for 7800 iterations: 74.14398151636124 mins
Train Loss: 0.32242095470428467
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 62.8% 20 way one-shot learning accuracy 


 ------------- 

Time for 8000 it

Got an average of 73.2% 20 way one-shot learning accuracy 


 ------------- 

Time for 14400 iterations: 134.24698913494746 mins
Train Loss: 0.26928141713142395
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 64.8% 20 way one-shot learning accuracy 


 ------------- 

Time for 14600 iterations: 135.99449942906696 mins
Train Loss: 0.14127105474472046
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 67.2% 20 way one-shot learning accuracy 


 ------------- 

Time for 14800 iterations: 137.7554041147232 mins
Train Loss: 0.18216948211193085
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 61.2% 20 way one-shot learning accuracy 


 ------------- 

Time for 15000 iterations: 139.5817441503207 mins
Train Loss: 0.15236537158489227
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 69.2% 20 way one-shot learning accuracy 


 ------------- 

Time for

In [154]:
best

78.0

In [221]:
#random testing (training set)
n=10
pairs,label=createPairs(X,y,sizes,n)
probs = model.predict(pairs)
count=0
for i in range(n):
    print("predicted : ",probs[i][0]," actual : ",label[i])
    if label[i]==1:
        if probs[i]>0.5:
            count+=1
    else:
        if probs[i]<0.5:
            count+=1 
        
print("acc = ",count/n) 

predicted :  0.9939139  actual :  1
predicted :  0.0  actual :  0
predicted :  0.9986825  actual :  1
predicted :  0.0  actual :  0
predicted :  0.98474383  actual :  1
predicted :  5.662441e-07  actual :  0
predicted :  0.98278785  actual :  1
predicted :  0.0003476739  actual :  0
predicted :  0.997195  actual :  1
predicted :  9.948859e-07  actual :  0
acc =  1.0


In [219]:
#random testing (testing set)
n=20
pairs,label=createPairs(Xval,yval,sizes_val,n)
probs = model.predict(pairs)
count=0
for i in range(n):
    print("predicted : ",probs[i][0]," actual : ",label[i])
    if label[i]==1:
        if probs[i][0]>0.5:
            count+=1
    else:
        if probs[i][0]<0.5:
            count+=1 
        
print("acc = ",count/n) 

predicted :  0.7119514  actual :  1
predicted :  0.0001386106  actual :  0
predicted :  0.7787981  actual :  1
predicted :  0.0  actual :  0
predicted :  0.7972748  actual :  1
predicted :  0.0070440173  actual :  0
predicted :  0.7438587  actual :  1
predicted :  0.00013965368  actual :  0
predicted :  0.0012342334  actual :  1
predicted :  9.23872e-07  actual :  0
predicted :  0.7537019  actual :  1
predicted :  1.630187e-05  actual :  0
predicted :  0.8445027  actual :  1
predicted :  0.0039846897  actual :  0
predicted :  0.9506608  actual :  1
predicted :  4.87864e-05  actual :  0
predicted :  0.7971395  actual :  1
predicted :  0.020821214  actual :  0
predicted :  0.5858815  actual :  1
predicted :  0.00061538815  actual :  0
acc =  0.95
